# Estimating Workplace Location

Integration with [larch](https://larch.newman.me) for model estimation. See [estimation tools review](https://github.com/ActivitySim/activitysim/wiki/Estimation-Tools-Review) for more information about larch.

# Run the Example

Output an estimation data bundle (EBD), which contains:
  - model settings - workplace_location_model_settings.yaml
  - coefficients - workplace_location_coefficients.csv
  - utilities specification - workplace_location_SPEC.csv
  - alternatives values - workplace_location_alternatives_combined.csv
  - chooser data - workplace_location_choosers_combined.csv
  - chooses made - workplace_location_choices.csv

# Read EDB 

In [1]:
import larch  # !conda install larch #for estimation
import os
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import larch_asim  # utility functions in a local module

In [2]:
edb_directory = "estimation_data_bundle/workplace_location/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

In [3]:
coefficients = read_csv("workplace_location_coefficients.csv", index_col='coefficient_name')
spec = read_csv("workplace_location_SPEC.csv")
alt_values = read_csv("workplace_location_alternatives_combined.csv")
chooser_data = read_csv("workplace_location_choosers_combined.csv")
choices = read_csv("workplace_location_choices.csv")
landuse = read_csv("workplace_location_landuse.csv", index_col='TAZ')
size_spec = read_csv("workplace_location_size_terms.csv")


In [4]:
landuse

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
TAZ,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,932.83514,0,0.00000,0.00000,0.00000,3,5.89564,2.875000,1707.375000,2.870167
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,885.61682,0,0.00000,0.00000,0.00000,1,5.84871,5.195214,1631.374751,5.178722
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,716.27252,0,0.00000,0.00000,0.00000,1,5.53231,80.470405,736.891913,72.547987
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,314.00000,0,0.00000,0.00000,0.00000,2,5.64330,7.947368,1180.736842,7.894233
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,314.01431,0,0.00000,72.14684,0.00000,1,5.52555,38.187500,978.875000,36.753679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,4,4,1,2779,8062,376.0,172.0,15.00000,1760,1178,...,0.00000,3,0.00000,0.00000,0.00000,1,2.04173,14.860963,9.411765,5.762347
187,4,4,1,1492,4139,214.0,116.0,10.00000,808,603,...,0.00000,3,0.00000,0.00000,0.00000,2,1.73676,11.841270,6.412698,4.159890
188,4,4,1,753,4072,232.0,11.0,178.00000,4502,1117,...,0.00000,2,3961.04761,17397.79102,11152.93652,1,2.28992,3.984127,23.820106,3.413233


## Work Size Spec (extraction)

In [5]:
work_size_spec = size_spec \
.query("model_selector == 'workplace'") \
.drop(columns='model_selector') \
.set_index('segment')
work_size_spec = work_size_spec.loc[:,work_size_spec.max()>0]
work_size_spec

,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN
segment,,,,,,
work_low,0.129,0.193,0.383,0.120,0.010,0.164
work_med,0.120,0.197,0.325,0.139,0.008,0.210
work_high,0.110,0.207,0.284,0.154,0.006,0.239
work_veryhigh,0.093,0.270,0.241,0.146,0.004,0.246


## Work Size Coefficients (extraction)

In [6]:
size_coef = work_size_spec.stack().reset_index()
size_coef.index = size_coef.iloc[:,0] +"_"+ size_coef.iloc[:,1]
size_coef = size_coef.loc[size_coef.iloc[:,2]>0]
size_coef['constrain'] = 'F'
one_each = size_coef.groupby('segment').first().reset_index()
size_coef.loc[one_each.iloc[:,0] +"_"+ one_each.iloc[:,1], 'constrain'] = 'T'
size_coef = size_coef.iloc[:,2:]
size_coef.columns = ['value','constrain']
size_coef.index.name = 'coefficient_name'
size_coef['value'] = np.log(size_coef['value'])
size_coef

,value,constrain
coefficient_name,,
work_low_RETEMPN,-2.047943,T
work_low_FPSEMPN,-1.645065,F
work_low_HEREMPN,-0.959720,F
work_low_OTHEMPN,-2.120264,F
work_low_AGREMPN,-4.605170,F
work_low_MWTEMPN,-1.807889,F
work_med_RETEMPN,-2.120264,T
work_med_FPSEMPN,-1.624552,F
work_med_HEREMPN,-1.123930,F


## Settings

In [7]:
settings = yaml.load(
    open(os.path.join(edb_directory,"workplace_location_model_settings.yaml"),"r"), 
    Loader=yaml.SafeLoader,
)

settings

{'SAMPLE_SIZE': 30,
 'SIMULATE_CHOOSER_COLUMNS': ['income_segment', 'TAZ'],
 'SAMPLE_SPEC': 'workplace_location_sample.csv',
 'SPEC': 'workplace_location.csv',
 'COEFFICIENTS': 'workplace_location_coeffs.csv',
 'LOGSUM_SETTINGS': 'tour_mode_choice.yaml',
 'LOGSUM_PREPROCESSOR': 'nontour_preprocessor',
 'LOGSUM_TOUR_PURPOSE': 'work',
 'CHOOSER_ORIG_COL_NAME': 'TAZ',
 'ALT_DEST_COL_NAME': 'alt_dest',
 'IN_PERIOD': 17,
 'OUT_PERIOD': 8,
 'DEST_CHOICE_COLUMN_NAME': 'workplace_taz',
 'annotate_persons': {'SPEC': 'annotate_persons_workplace',
  'DF': 'persons',
  'TABLES': ['land_use']},
 'annotate_households': {'SPEC': 'annotate_households_workplace',
  'DF': 'households',
  'TABLES': ['persons']},
 'CHOOSER_TABLE_NAME': 'persons_merged',
 'MODEL_SELECTOR': 'workplace',
 'CHOOSER_SEGMENT_COLUMN_NAME': 'income_segment',
 'CHOOSER_FILTER_COLUMN_NAME': 'is_worker',
 'SEGMENT_IDS': {'work_low': 1,
  'work_med': 2,
  'work_high': 3,
  'work_veryhigh': 4},
 'CONSTANTS': {'WORK_HIGH_SEGMENT_ID': 3

## Coefficients

In [8]:
coefficients

,value,constrain
coefficient_name,,
coef_dist_0_1,-0.8428,F
coef_dist_1_2,-0.3104,F
coef_dist_2_5,-0.3783,F
coef_dist_5_15,-0.1285,F
coef_dist_15_up,-0.0917,F
coef_dist_0_5_high,0.1500,F
coef_dist_5_up_high,0.0200,F
coef_mode_logsum,0.3000,F


## Spec

In [9]:
spec

,Label,Description,Expression,coefficient
0,local_dist,NaN,_DIST@skims['DIST'],1
1,util_dist_0_1,"Distance, piecewise linear from 0 to 1 miles","@_DIST.clip(0,1)",coef_dist_0_1
2,util_dist_1_2,"Distance, piecewise linear from 1 to 2 miles","@(_DIST-1).clip(0,1)",coef_dist_1_2
3,util_dist_2_5,"Distance, piecewise linear from 2 to 5 miles","@(_DIST-2).clip(0,3)",coef_dist_2_5
4,util_dist_5_15,"Distance, piecewise linear from 5 to 15 miles","@(_DIST-5).clip(0,10)",coef_dist_5_15
5,util_dist_15_up,"Distance, piecewise linear for 15+ miles",@(_DIST-15.0).clip(0),coef_dist_15_up
6,util_dist_0_5_high,"Distance 0 to 5 mi, high and very high income",@(df['income_segment']>=WORK_HIGH_SEGMENT_ID) ...,coef_dist_0_5_high
7,util_dist_15_up_high,"Distance 5+ mi, high and very high income",@(df['income_segment']>=WORK_HIGH_SEGMENT_ID) ...,coef_dist_5_up_high
8,util_size_variable,Size variable,@(df['size_term'] * df['shadow_price_size_term...,1
9,util_utility_adjustment,utility adjustment,@df['shadow_price_utility_adjustment'],1


In [10]:
# Remove shadow pricing and pre-existing size expression

spec = spec\
.set_index('Label')\
.drop(index=['util_size_variable', 'util_utility_adjustment'])\
.reset_index()

## Alt Values

In [11]:
alt_values

,person_id,variable,1,2,3,4,5,6,7,8,...,181,182,183,184,185,186,187,188,189,190
0,110274,TAZ,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,181.0,182.0,183.0,184.0,185.0,186.0,187.0,188.0,189.0,190.0
1,110274,mode_choice_logsum,-0.47944851909784375,-0.28552521143267007,-0.30761367235529846,-0.24837207220778174,-0.2027951273777935,-0.5146191347308435,-0.3942615298576022,0.10708464401436528,...,-0.8152745809007619,-0.9352596221642304,-0.8396785283151357,-0.7454278382169685,-0.6642367603325747,-0.7360307976703867,-0.8922738873708044,-0.7536225578783141,-0.8524490550078891,-1.0103232877706299
2,110274,pick_count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,110274,prob,0.057150840242627544,0.08469690499898065,0.005246768453568192,0.05174135498396009,0.036268359625744574,0.006904411205017171,0.022177275014310764,0.006808200370770835,...,8.074495966703502e-05,7.572843672313463e-05,0.0003166497489420263,0.00037108729605378374,0.0005699251275273905,0.00046081884065046075,0.0001799514663573611,0.0012246927616559123,5.2129276044611955e-05,0.00031078806541221486
4,110274,shadow_price_size_term_adjustment,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,7348239,util_mode_logsum,-1.1618321687208726,-1.0036209190477476,-1.6443615938430654,-0.9985735768846415,-1.618989524235128,-1.7269280452466291,-1.6705924063455906,-1.0125002778100034,...,-0.8786959280980746,-0.7551187435158331,-0.6818291367725213,-0.7301824836938693,-0.6588210025797607,-0.6218341269365818,-0.5424107164994928,-0.5237309882940743,-0.6440006321969093,-0.6918554535160348
2523,7348239,util_no_attractions,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2524,7348239,util_sample_of_corrections_factor,3.3747705061837348,2.934932920568512,5.829959008655943,3.568024502775243,4.020399259983128,5.443360043508121,4.2410951554815,5.283915342569453,...,8.246769750329333,8.190553778588678,6.976066460838195,6.66636517375826,6.327797452252676,6.407835558942262,7.315643685720587,5.498201483963222,8.333471316031241,6.902273720621597
2525,7348239,util_size_variable,8.849766838403042,9.283094427998961,6.380473322824815,8.62938784109069,8.165078121147992,6.750797329123684,7.9273160630170745,6.838835976217513,...,4.25552759981349,4.218433599118909,5.401455915465108,5.687287172127004,5.965094904087839,5.903501808626576,4.9316786630945435,6.695955888668583,3.939891058770619,5.4079786207575555


## Chooser Data

In [12]:
chooser_data

,person_id,income_segment,TAZ,model_choice,override_choice
0,110274,1,13,164,2
1,112798,1,19,160,19
2,115213,1,40,36,101
3,121226,1,88,123,148
4,125886,1,113,131,17
...,...,...,...,...,...
128,6368322,4,101,132,2
129,6918939,4,121,141,15
130,6918940,4,121,169,9
131,6918943,4,121,6,9


## Choices

In [13]:
choices

,person_id,model_choice
0,110274,164
1,112798,160
2,115213,36
3,121226,123
4,125886,131
...,...,...
261,6368322,132
262,6918939,141
263,6918940,169
264,6918943,6


# Data Setup

In [14]:
from larch import P, X

In [15]:
m = larch.Model()

In [16]:
m.utility_ca = larch_asim.linear_utility_from_spec(
    spec, x_col='Label', p_col='coefficient', 
    ignore_x=('local_dist',), 
)
print(m.utility_ca)

  P.coef_dist_0_1 * X.util_dist_0_1
+ P.coef_dist_1_2 * X.util_dist_1_2
+ P.coef_dist_2_5 * X.util_dist_2_5
+ P.coef_dist_5_15 * X.util_dist_5_15
+ P.coef_dist_15_up * X.util_dist_15_up
+ P.coef_dist_0_5_high * X.util_dist_0_5_high
+ P.coef_dist_5_up_high * X.util_dist_15_up_high
+ P('-999') * X.util_no_attractions
+ P.coef_mode_logsum * X.util_mode_logsum
+ P('1') * X.util_sample_of_corrections_factor


In [17]:
m.quantity_ca = sum(
    P(f"{i}_{q}") * X(q) * X(f"income_segment=={settings['SEGMENT_IDS'][i]}")
    for i in work_size_spec.index
    for q in work_size_spec.columns
)

In [18]:
larch_asim.apply_coefficients(coefficients, m)
larch_asim.apply_coefficients(size_coef, m, minimum=-6, maximum=6)

In [19]:
m.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,
1,1.000000,1.0,1.0,1.0,1.0,1,
coef_dist_0_1,-0.842800,0.0,0.0,NaN,NaN,0,
coef_dist_0_5_high,0.150000,0.0,0.0,NaN,NaN,0,
coef_dist_15_up,-0.091700,0.0,0.0,NaN,NaN,0,
coef_dist_1_2,-0.310400,0.0,0.0,NaN,NaN,0,
coef_dist_2_5,-0.378300,0.0,0.0,NaN,NaN,0,
coef_dist_5_15,-0.128500,0.0,0.0,NaN,NaN,0,
coef_dist_5_up_high,0.020000,0.0,0.0,NaN,NaN,0,
coef_mode_logsum,0.300000,0.0,0.0,NaN,NaN,0,


In [20]:
x_co = chooser_data.set_index('person_id').rename(columns={'TAZ':'HOMETAZ'})

In [21]:
x_ca = larch_asim.cv_to_ca(
    alt_values.set_index(['person_id', 'variable'])
)

In [22]:
x_ca_1 = pd.merge(x_ca, landuse, on='TAZ', how='left')
x_ca_1.index = x_ca.index

In [23]:
d = larch.DataFrames(
    co=x_co,
    ca=x_ca_1,
    av=True,
)

In [24]:
d.info(1)

larch.DataFrames:  (not computation-ready)
  n_cases: 133
  n_alts: 190
  data_ca:
    - TAZ                               (25270 non-null float64)
    - mode_choice_logsum                (25270 non-null float64)
    - pick_count                        (25270 non-null float64)
    - prob                              (25270 non-null float64)
    - shadow_price_size_term_adjustment (25270 non-null float64)
    - shadow_price_utility_adjustment   (25270 non-null float64)
    - size_term                         (25270 non-null float64)
    - util_dist_0_1                     (25270 non-null float64)
    - util_dist_0_5_high                (25270 non-null float64)
    - util_dist_15_up                   (25270 non-null float64)
    - util_dist_15_up_high              (25270 non-null float64)
    - util_dist_1_2                     (25270 non-null float64)
    - util_dist_2_5                     (25270 non-null float64)
    - util_dist_5_15                    (25270 non-null float64)
    - u

In [25]:
m.dataservice = d

In [26]:
m.choice_co_code = 'override_choice'

# Estimate

Note: The demo test data here is 100 households and the model has 
57 estimated parameters -- the result is a very over-specified
model which does not have a numerically stable likelihood maximizing
solution.

In [27]:
m.estimate()

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
coef_dist_0_1,-1.389352,0.0,0.0,NaN,NaN,0,,-1.389352
coef_dist_0_5_high,0.270129,0.0,0.0,NaN,NaN,0,,0.270129
coef_dist_15_up,-0.091700,0.0,0.0,NaN,NaN,0,,-0.091700
coef_dist_1_2,-0.551071,0.0,0.0,NaN,NaN,0,,-0.551071
coef_dist_2_5,-0.617273,0.0,0.0,NaN,NaN,0,,-0.617273
coef_dist_5_15,-0.170816,0.0,0.0,NaN,NaN,0,,-0.170816
coef_dist_5_up_high,-0.180773,0.0,0.0,NaN,NaN,0,,-0.180773
coef_mode_logsum,0.251960,0.0,0.0,NaN,NaN,0,,0.251960


/Users/jeffnewman/opt/anaconda3/envs/garage37/lib/python3.7/site-packages/ipykernel_launcher.py:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  """Entry point for launching an IPython kernel.
/Users/jeffnewman/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.025413600137196e-14 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
/Users/jeffnewman/opt/anaconda3/envs/garage37/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


┣                x: -999                    -999.000000
┃                   1                          1.000000
┃                   coef_dist_0_1             -1.389352
┃                   coef_dist_0_5_high         0.270129
┃                   coef_dist_15_up           -0.091700
┃                   coef_dist_1_2             -0.551071
┃                   coef_dist_2_5             -0.617273
┃                   coef_dist_5_15            -0.170816
┃                   coef_dist_5_up_high       -0.180773
┃                   coef_mode_logsum           0.251960
┃                   work_high_AGREMPN         -5.123732
┃                   work_high_FPSEMPN          6.000000
┃                   work_high_HEREMPN          2.792371
┃                   work_high_MWTEMPN         -2.304052
┃                   work_high_OTHEMPN         -4.535634
┃                   work_high_RETEMPN         -2.207275
┃                   work_low_AGREMPN          -4.611383
┃                   work_low_FPSEMPN           5.210360
┃                   work_low_HEREMPN           3.080240
┃                   work_low_MWTEMPN           6.000000
┃                   work_low_OTHEMPN          -5.543854
┃                   work_low_RETEMPN          -2.047943
┃                   work_med_AGREMPN          -4.820800
┃                   work_med_FPSEMPN           6.000000
┃                   work_med_HEREMPN          -1.872117
┃                   work_med_MWTEMPN          -3.610164
┃                   work_med_OTHEMPN           2.517801
┃                   work_med_RETEMPN          -2.120264
┃                   work_veryhigh_AGREMPN     -5.527867
┃                   work_veryhigh_FPSEMPN      6.000000
┃                   work_veryhigh_HEREMPN     -5.105099
┃                   work_veryhigh_MWTEMPN     -4.180978
┃                   work_veryhigh_OTHEMPN     -5.605457
┃                   work_veryhigh_RETEMPN     -2.375156
┃                   dtype: float64
┣          loglike: -647.9729549038722
┣        d_loglike: -999                     0.000000e+00
┃                   1                        0.000000e+00
┃                   coef_dist_0_1            1.507885e-04
┃                   coef_dist_0_5_high       3.125795e-04
┃                   coef_dist_15_up          0.000000e+00
┃                   coef_dist_1_2            6.282337e-04
┃                   coef_dist_2_5           -6.228366e-04
┃                   coef_dist_5_15          -1.241786e-03
┃                   coef_dist_5_up_high     -9.129772e-04
┃                   coef_mode_logsum        -3.179356e-04
┃                   work_high_AGREMPN       -1.499370e-06
┃                   work_high_FPSEMPN        5.209772e-03
┃                   work_high_HEREMPN       -2.215770e-04
┃                   work_high_MWTEMPN       -1.009602e-04
┃                   work_high_OTHEMPN       -1.644618e-04
┃                   work_high_RETEMPN        0.000000e+00
┃                   work_low_AGREMPN        -6.599647e-07
┃                   work_low_FPSEMPN        -5.805328e-05
┃                   work_low_HEREMPN        -3.541263e-05
┃                   work_low_MWTEMPN         2.672785e-03
┃                   work_low_OTHEMPN        -7.831623e-05
┃                   work_low_RETEMPN         0.000000e+00
┃                   work_med_AGREMPN         1.281445e-06
┃                   work_med_FPSEMPN         3.631694e-03
┃                   work_med_HEREMPN        -2.322541e-04
┃                   work_med_MWTEMPN        -1.119334e-04
┃                   work_med_OTHEMPN         2.029143e-05
┃                   work_med_RETEMPN         0.000000e+00
┃                   work_veryhigh_AGREMPN   -1.291536e-06
┃                   work_veryhigh_FPSEMPN    4.188257e-03
┃                   work_veryhigh_HEREMPN   -1.053989e-04
┃                   work_veryhigh_MWTEMPN   -1.225766e-04
┃                   work_veryhigh_OTHEMPN   -1.291323e-04
┃                   work_veryhigh_RETEMPN    0.000000e+00
┃                   dtype: float64
┣              

In [28]:
# m.possible_overspecification

# Outputs

In [29]:
est_names = [j for j in coefficients.index if j in m.pf.index]
coefficients.loc[est_names,'value'] = m.pf.loc[est_names, 'value']

In [30]:
# Write out replacement coefficients file and model summaries
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

In [31]:
coefficients.reset_index().to_csv(
    os.path.join(edb_directory,'estimated',"workplace_location_coefficients_revised.csv"), 
    index=False,
)

In [32]:
m.to_xlsx(
    os.path.join(edb_directory,'estimated',"workplace_location_model_estimation.xlsx"), 
)

In [33]:
# Write size coefficients into size_spec
for c in work_size_spec.columns:
    for i in work_size_spec.index:
        param_name = f"{i}_{c}"
        j = (size_spec['segment'] == i) & (size_spec['model_selector'] == 'workplace')
        size_spec.loc[j,c] = np.exp(m.get_value(param_name))
        

In [34]:
# Rescale each row to total 1, not mathematically needed
# but to maintain a consistent approach from existing ASim

size_spec.iloc[:,2:] = (size_spec.iloc[:,2:].div(size_spec.iloc[:,2:].sum(1), axis=0))

In [35]:
size_spec.to_csv(
    os.path.join(edb_directory,'estimated',"workplace_location_size_terms.csv"), 
    index=False,
)